In [4]:
%run excute_nonZS_MaxMin.ipynb
import torch
from torch_geometric.loader import DataLoader
from torch_geometric.data import Dataset
from torch.nn import Linear, Module, Sequential
import torch.nn.functional as F
from torch_geometric.nn import GCNConv


C:\Users\xzhan176\Anaconda3\lib\site-packages\nbformat\__init__.py:92: MissingIDFieldWarning: Code cell is missing an id field, this will become a hard error in future nbformat versions. You may want to use `normalize()` on your notebooks before validations (available since nbformat 5.1.4). Previous versions of nbformat are fixing this issue transparently, and will stop doing so in the future.
  validate(nb)


In [ ]:
# create the data
adj_list = []
feat_list = []
labels_list = []
for i in range(10):
    (v1,v2, max_opinion, min_opinion,max_pol, s, G, a,b,c,d) = run(n)
    adj_list.append(torch.tensor(G))
    feat_list.append(torch.tensor(s))
    #create min_action label
    c1 = [0 for i in range(20)]
    c1[v2]=1
    print(c1)
    labels= c1
    labels_list.append(torch.tensor(labels))


In [1]:
# step1: Load graph data for each social network and process it
# ...

# Merge adjacency matrices and node features into a single tensor
adj = torch.stack(adj_list, dim=0)  # shape: (n, num_nodes, num_nodes)
feat = torch.stack(feat_list, dim=0)  # shape: (n, num_nodes, num_features)
labels = torch.stack(labels_list, dim=0)  # shape: (n, num_nodes)



NameError: name 'torch' is not defined

In [91]:
# Create graph indicator tensor
graph_indicator = torch.zeros(adj.shape[:2], dtype=torch.long)
for i, adj_i in enumerate(adj_list):
    graph_indicator[i, :adj_i.shape[0]] = i


torch.Size([10, 20])


In [152]:
# Create batch size tensor
batch_size = n #torch.tensor([adj_i.shape[0] for adj_i in adj_list], dtype=torch.long)

# Define dataset class
class GraphDataset(Dataset):
    def __init__(self, adj, feat, labels, graph_indicator):
        self.adj = adj
        self.feat = feat
        self.labels = labels
        self.graph_indicator = graph_indicator
        # Compute the number of node features
        self.num_node_features = feat.shape[1] #number of features
        
    def __len__(self):
        return self.adj.shape[0]
    
    def __getitem__(self, idx):
        return self.adj[idx], self.feat[idx], self.labels[idx], self.graph_indicator[idx]
    
# Create DataLoader
num_classes = 2
hidden_channels = 32
dataset = GraphDataset(adj, feat, labels, graph_indicator)
#dataloader = DataLoader(dataset, batch_size=batch_size, collate_fn=lambda x: x)
# Split the data into training and validation sets
train_data, test_data = torch.utils.data.random_split(dataset, [int(0.8*len(dataset)), len(dataset)-int(0.8*len(dataset))])
# Create data loaders for batch processing
train_loader = DataLoader(train_data, batch_size=1, shuffle=True) 
test_loader = DataLoader(test_data, batch_size=1, shuffle=True)#collate_fn=lambda x: x)

#Step 2: Define the GCN model
#Define GCN model with graph-level batching
###################################################################
class GCN(torch.nn.Module):
    def __init__(self, feat, num_classes):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(feat.shape[1], 64)
        self.conv2 = GCNConv(64, num_classes)
        self.fc1 = torch.nn.Linear(feat.shape[1], 32)
        self.fc2 = torch.nn.Linear(32, num_classes)

    def forward(self, x, edge_index, graph_indicator):
        # Perform convolutional layers
        x = F.relu(self.conv1(x, edge_index))
        x = F.relu(self.conv2(x, edge_index))
        # Pool across the nodes in each graph
        x = global_max_pool(x, graph_indicator)
        # Perform fully connected layers
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        # Apply log_softmax to get log probabilities
        x = F.log_softmax(x, dim=1)
        return x
# #_____________________________________________________________________
# # Define the GCN model
# class GCN(torch.nn.Module):
#     def __init__(self, num_features, num_classes):
#         super(GCN, self).__init__()
#         self.conv1 = GCNConv(num_features, 32)
#         self.conv2 = GCNConv(32, num_classes)

#     def forward(self, x, edge_index):
#         x = self.conv1(x, edge_index)
#         x = F.relu(x)
#         x = F.dropout(x, training=self.training)
#         x = self.conv2(x, edge_index)
#         return F.log_softmax(x, dim=1)

In [153]:
def test(test_loader):
    # Set model to evaluation mode
    model.eval()

    # Initialize variables for computing test loss
    test_loss = 0.
    num_correct = 0

    # Loop over each batch of graphs
    for data in test_loader:
        print("data")
        print(data)
        # Get the adjacency matrix, node features, and target labels for the current batch
        adj_list, features_list, target_list, graph_indicator = data.adj, data.feat, data.labels, data.graph_indicator

        # Forward pass
        output_list = model(adj_list, features_list,graph_indicator)

        # Compute loss
        loss = 0
        for i in range(len(output_list)):
            loss += F.nll_loss(output_list[i], target_list[i])
        loss /= len(output_list)

        # Update variables for computing test loss
        test_loss += loss.item() * len(target_list)
        num_correct += count_correct(output_list, target_list)

    # Compute average test loss and accuracy
    test_loss /= len(test_loader.dataset)
    accuracy = num_correct / len(test_loader.dataset)

    return test_loss

def train(num_epochs, train_loader, test_loader):
    for epoch in range(num_epochs):
        # Set model to training mode
        model.train()

        # Loop over each batch of graphs
        for data in train_loader:#enumerate(train_loader):
            print("data")
            print(data)
            # Get the adjacency matrix, node features, and target labels for the current batch
            adj_list, features_list, target_list, graph_indicator = data#data.adj, data.feat, data.labels, data.graph_indicator

            # Zero out the gradients
            optimizer.zero_grad()

            # Forward pass
            output_list = model(adj_list, features_list, graph_indicator)

            # Compute loss
            loss = 0
            for i in range(len(output_list)):
                loss += F.nll_loss(output_list[i], target_list[i])
            loss /= len(output_list)

            # Backward pass
            loss.backward()

            # Update parameters
            optimizer.step()

        # Evaluate the model on the test set
        test_loss = test(test_loader)

        # Print loss for the current epoch
        print('Epoch: {:02d}, Train Loss: {:.4f}, Test Loss: {:.4f}'.format(epoch, loss.item(), test_loss))





In [154]:
# Step 5: Train and test the GCN model for multiple epochs
model = GCN(dataset.feat, num_classes)

optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
train_losses = []
test_losses = []
for epoch in range(20):
    train_loss = train(epoch, train_loader, test_loader)
    train_losses.append(train_loss)
#     test_acc = test(test_loader)
#     test_losses.append(test_acc)
#     print(f'Epoch: {epoch:03d}, Train Loss: {train_loss:.4f}, Test Acc: {test_acc:.4f}')

# Step 6: Plot the training and testing loss over time
plt.plot(train_losses, label='Training Loss')
plt.plot(test_losses, label='Testing Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Value')
plt.legend()
plt.show()

data
[tensor([[[0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
         [1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0],
         [1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0],
         [1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0],
         [1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0],
         [1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0],
         [1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0],
         [0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0],
         [0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0],
         [1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0],
         [0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0],
         [0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0],
         [0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0],
         [0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0],


IndexError: index 1 is out of bounds for dimension 0 with size 1